In [ ]:
%load_ext autoreload
%autoreload 2
from use_case.tests import * 
from models.eval import *
from models.model import *
from models.trainer import *
from auto_tests.hypernet_test import * 
from auto_tests.abm_test import *
from models.trainer import load_model
import matplotlib.pyplot as plt


# Hypernet Test

In [ ]:
def plot_results(data, title_prefix, model_name, result_type="Returns"):
    fig, axs = plt.subplots(2, 5, figsize=(30, 12))
    fig.suptitle(f"{result_type} Distributions - {model_name}", fontsize=24)

    for i in range(10):
        ax = axs[i // 5, i % 5]
        ax.hist(data[i], bins=15, alpha=0.7, color='skyblue', edgecolor='black')
        ax.set_title(f'Type {i}', fontsize=16)
        ax.set_xlabel(result_type, fontsize=14)
        ax.set_ylabel('Frequency', fontsize=14)
        ax.tick_params(axis='both', labelsize=12)

    plt.tight_layout(rect=[0, 0, 1, 0.95])
    plt.savefig(f"{model_name}_{result_type.lower()}.png", dpi=300)
    plt.close()

def compute_action_entropy(histogram_data, num_actions):
    action_counts = np.bincount(histogram_data, minlength=num_actions)
    total_actions = np.sum(action_counts)
    if total_actions == 0:
        return 0.0
    probabilities = action_counts / total_actions
    probabilities = probabilities[probabilities > 0]
    entropy = -np.sum(probabilities * np.log(probabilities))
    return entropy

env = initialize_baseline_hetero()
parameters = ParameterSettings(
    n_agents=env.n_agents,
    d_action=env.n_actions,
    d_obs=env.obs_size,
    d_traits=env.d_traits,
    d_het_latent=4,
    d_beliefs=env.d_beliefs,
    d_relation=env.d_relation,
    d_message=4,
    d_comm_state=env.d_comm_state,
    device="cuda"
)

MODELS_DIR = "foo\\h models"
OUTPUT_DIR = "output_data\\heterogeneous"

run_idx = 1
for model_file in os.listdir(MODELS_DIR):
    if model_file.endswith(".pt"):
        file_path = os.path.join(MODELS_DIR, model_file)

        try:
            model = PPOModel(parameters)
            model = load_model(model, file_path)
            model.eval()

            _, hist = hypernet_test(model, env, 10, -1)

            run_idx = model_file.split('.')[0]
            fig, axs = plt.subplots(2, 5, figsize=(30, 16))
            fig.suptitle(f"Heterogeneous Environment Action Distribution: Run {run_idx}", fontsize=24)

            for type_idx in range(10):
                row, col = divmod(type_idx, 5)
                actions = hist[type_idx]
                entropy = compute_action_entropy(actions, 10)
                axs[row, col].hist(
                    actions,
                    bins=10,
                    density=True,
                    alpha=0.7,
                    edgecolor='black'
                )
                axs[row, col].set_title(f"Type {type_idx}", fontsize=20)
                axs[row, col].set_xlabel("Action", fontsize=16)
                axs[row, col].set_ylabel("Frequency", fontsize=16)
                axs[row, col].set_xticks(range(env.n_actions))
                axs[row, col].tick_params(axis='both', labelsize=14)
                axs[row, col].grid(axis='y', alpha=0.5)
                axs[row, col].text(0.5, -0.2, f'Entropy: {entropy:.2f}',
                                   horizontalalignment='center', verticalalignment='top',
                                   transform=axs[row, col].transAxes, fontsize=18)

            plt.tight_layout(rect=[0, 0, 1, 0.95])
            plt.savefig(f"{model_file}.jpg", dpi=300)
            plt.close()

        except Exception as e:
            print(f"Error processing {model_file}: {str(e)}")


# Inference Test

In [19]:
import os
from tensorboard.backend.event_processing import event_accumulator
import matplotlib.pyplot as plt
import numpy as np

# log_root = "runs\\baseline_test_type_net"
# distribution_name = "Eval/agent_total_rewards"
# OUTPUT_DIR = "output_data\\type_net"

# log_root = "runs\\batch_test_type_inf"
# distribution_name = "Eval/agent_total_rewards"
# OUTPUT_DIR = "output_data\\type_inf"


# log_root = "runs\\batch_test_baseline"
# distribution_name = "Eval/action_distribution"
# OUTPUT_DIR = "output_data\\baseline_dists"

log_root = "runs\\baseline_test_type_net"
distribution_name = "Eval/action_distribution"
OUTPUT_DIR = "output_data\\type_net_dists"


os.makedirs(OUTPUT_DIR, exist_ok=True)

subfolders = [os.path.join(log_root, f) for f in os.listdir(log_root) if os.path.isdir(os.path.join(log_root, f))]

for folder in subfolders:
    events_files = [os.path.join(folder, f) for f in os.listdir(folder) if f.startswith("events.out.tfevents")]
    if not events_files:
        continue

    run_idx = os.path.basename(folder)

    ea = event_accumulator.EventAccumulator(folder, size_guidance={event_accumulator.HISTOGRAMS: 1000})
    ea.Reload()

    if distribution_name not in ea.Tags().get('histograms', []):
        continue

    histograms = ea.Histograms(distribution_name)
    latest = histograms[-1]  # most recent histogram
    histogram = latest.histogram_value

    bucket_limits = np.array(latest.histogram_value.bucket_limit)
    bucket_counts = np.array(latest.histogram_value.bucket)

    # Prepend a lower bound to get left/right edges of buckets
    lower_bound = bucket_limits[0] - 1.0  # or use -np.inf
    edges = np.concatenate([[lower_bound], bucket_limits])

    # Compute midpoints for each bucket
    mids = (edges[:-1] + edges[1:]) / 2

    # Now safely repeat each midpoint value according to the count in that bucket
    data = np.repeat(mids, bucket_counts.astype(int))


    if len(data) == 0:
        continue

    median = np.median(data)
    min = np.min(data)
    max = np.max(data)
    bins = int(max - min) + 2

    plt.figure(figsize=(10, 6))
    plt.hist(data, bins=5, color='skyblue', edgecolor='black', density=False)
    # plt.axvline(median, color='red', linestyle='dashed', linewidth=1.5, label=f"Median: {median:.2f}")
    plt.title(f"Action distribution for the Network Environment Run {run_idx}")
    # plt.legend()
    output_path = os.path.join(OUTPUT_DIR, f"{run_idx}.jpg")
    plt.savefig(output_path, dpi=300)
    plt.close()


# Time Series Plots

In [ ]:
import os
from tensorboard.backend.event_processing import event_accumulator
import matplotlib.pyplot as plt
import numpy as np

# ===== USER CONFIGURATION =====
# log_root = "runs\\batch_test_baseline"
# scalar_names = ["Eval/median_rewards",]  # List of scalars to plot
# OUTPUT_DIR = "output_data\\baseline"

# OUTPUT_DIR = "output_data"
# log_root = "runs\\s\\hypernet-8000"

# OUTPUT_DIR = "output_data"
# log_root = "runs\\y"

# OUTPUT_DIR = "output_data\\baseline_loss"
# scalar_names = ["Actor/Metrics/Actor Loss", "Actor/Metrics/Critic Loss"]
# log_root = "runs\\batch_test_baseline"

OUTPUT_DIR = "output_data\\het_median rewards"
scalar_names =  ["Eval/median_rewards",]
log_root = "runs\\baseline_het"
# ==============================

os.makedirs(OUTPUT_DIR, exist_ok=True)
subfolders = [os.path.join(log_root, f) for f in os.listdir(log_root) 
              if os.path.isdir(os.path.join(log_root, f))]

for folder in subfolders:
    events_files = [os.path.join(folder, f) for f in os.listdir(folder) 
                   if f.startswith("events.out.tfevents")]
    if not events_files:
        continue

    run_idx = os.path.basename(folder)
    
    # Configure to load scalars efficiently
    ea = event_accumulator.EventAccumulator(
        folder,
        size_guidance={event_accumulator.SCALARS: 10000}
    )
    ea.Reload()
    
    # Create figure with subplots for each scalar
    n_scalars = len(scalar_names)
    fig, axes = plt.subplots(n_scalars, 1, figsize=(10, 4 * n_scalars), sharex=True)
    
    # If only one scalar, convert axes to list for consistent processing
    if n_scalars == 1:
        axes = [axes]
    
    # Track if we found any valid scalars
    found_any = False
    
    for i, scalar_name in enumerate(scalar_names):
        # Check if scalar exists in this run
        if scalar_name not in ea.Tags().get('scalars', []):
            print(f"Scalar '{scalar_name}' not found in run: {run_idx}")
            continue
            
        found_any = True
        # Extract all scalar events
        scalar_events = ea.Scalars(scalar_name)
        steps = [event.step for event in scalar_events]
        values = [event.value for event in scalar_events]
        
        # Plot to the current subplot
        ax = axes[i]
        ax.plot(steps, values, 'b-', linewidth=1.5)
        ax.set_ylabel("Value")
        ax.grid(True, alpha=0.3)
    
    if not found_any:
        plt.close(fig)
        print(f"No valid scalars found for run: {run_idx}")
        continue
    
    # Set common x-label at bottom
    axes[-1].set_xlabel("Training Step", labelpad=10)
    
    # Set main title for the entire figure with more space
    fig.suptitle(f"Median rewards observed in the Heterogeneous Environment. Run {run_idx}", fontsize=16, y=0.98)
    
    # Adjust layout with more space at the top
    plt.tight_layout()
    plt.subplots_adjust(top=0.92)  # Increased space above subplots
    
    # Save plot
    output_path = os.path.join(OUTPUT_DIR, f"{run_idx}.jpg")
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close(fig)
    print(f"Saved plot for run {run_idx} to {output_path}")

Saved plot for run 1 to output_data\het_median rewards\1.jpg
Saved plot for run 10 to output_data\het_median rewards\10.jpg


KeyboardInterrupt: 

In [4]:
import os
from tensorboard.backend.event_processing import event_accumulator
import matplotlib.pyplot as plt
import numpy as np

# ===== USER CONFIGURATION =====
OUTPUT_DIR = "output_data\\baseline_loss"
scalar_names = ["Actor/Metrics/Actor Loss", "Actor/Metrics/Critic Loss"]
log_root = "runs\\batch_test_baseline"
# ==============================

os.makedirs(OUTPUT_DIR, exist_ok=True)
subfolders = [os.path.join(log_root, f) for f in os.listdir(log_root) 
              if os.path.isdir(os.path.join(log_root, f))]

# Create one figure per scalar (each with a single plot containing all runs)
for scalar_name in scalar_names:
    # Create a new figure for this scalar
    plt.figure(figsize=(10, 6))
    
    # Track if we have any data for this scalar
    found_any = False
    
    for folder in subfolders:
        run_idx = os.path.basename(folder)
        events_files = [os.path.join(folder, f) for f in os.listdir(folder) 
                       if f.startswith("events.out.tfevents")]
        if not events_files:
            continue
            
        # Load TensorBoard data
        ea = event_accumulator.EventAccumulator(
            folder,
            size_guidance={event_accumulator.SCALARS: 10000}
        )
        ea.Reload()
        
        # Check if scalar exists
        if scalar_name not in ea.Tags().get('scalars', []):
            print(f"Scalar '{scalar_name}' not found in run: {run_idx}")
            continue
            
        found_any = True
        # Extract scalar data
        scalar_events = ea.Scalars(scalar_name)
        steps = [event.step for event in scalar_events]
        values = [event.value for event in scalar_events]
        
        # Plot this run's data with run_idx as label
        plt.plot(steps, values, label=run_idx, linewidth=1.0)
    
    if not found_any:
        plt.close()
        print(f"No data found for scalar '{scalar_name}' in any run.")
        continue
        
    # Configure plot appearance
    plt.title("Consolidated Plot of Actor (Top) and Critic (Bottom) Loss for models on the Simple Environment")
    plt.xlabel("Training Step")
    plt.ylabel("Value")
    plt.grid(True, alpha=0.3)
    plt.legend(loc='best', ncol=2, fontsize='x-small')  # Compact multi-column legend
    
    # Save scalar plot
    safe_name = scalar_name.replace("/", "_")
    output_path = os.path.join(OUTPUT_DIR, f"{safe_name}.jpg")
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()
    print(f"Saved merged plot for scalar '{scalar_name}' to {output_path}")

Saved merged plot for scalar 'Actor/Metrics/Actor Loss' to output_data\baseline_loss\Actor_Metrics_Actor Loss.jpg
Saved merged plot for scalar 'Actor/Metrics/Critic Loss' to output_data\baseline_loss\Actor_Metrics_Critic Loss.jpg


# ABM Test

In [ ]:
import matplotlib.pyplot as plt

OUTPUT_DIR = "output_data\\abm"
os.makedirs(OUTPUT_DIR, exist_ok=True)

env = initialize_sir_env()
parameters = ParameterSettings(
    n_agents=env.n_agents,
    d_action=env.n_actions,
    d_obs=env.obs_size,
    d_traits=env.d_traits,
    d_het_latent=4,
    d_beliefs=env.d_beliefs,
    d_relation=env.d_relation,
    d_message=4,
    d_comm_state=env.d_comm_state,
    device="cuda"
)

for i in range(0, 20): 
    plt.figure(figsize=(10, 6))
    checkpoint_steps = [35, 60, 120, 200]
    colors = ['red', 'blue', 'green', 'purple']  # Different color for each checkpoint

    seed = int(np.random.uniform(0, 100000))
    for step, color in zip(checkpoint_steps, colors):
        # Load model and run evaluation
        model = PPOModel(parameters)
        model = load_model(model, f"foo\\si model\\checkpoint_ppo_step_{step}.pt")
        env = initialize_sir_env(seed=seed, eps_length=100)
        _, all_episode_stats = si_eval_loop(model, env, 1, 1.0)
        
        # Extract infection data
        episode_stats = all_episode_stats[0]
        timesteps = range(len(episode_stats))
        infected_counts = [s['infected'] for s in episode_stats]
        
        # Plot with checkpoint-specific label and color
        plt.plot(timesteps, infected_counts, marker='o', linestyle='-', 
                label=f'Step {step}', color=color)

    # Configure unified plot
    plt.title(f'Infection Over Time on the SI Environment Run {i+1}')
    plt.xlabel('Timestep')
    plt.ylabel('Infected Individuals')
    plt.grid(True, alpha=0.3)
    plt.legend(title='Checkpoint Step')
    plt.tight_layout()
    
    # Save plot with iteration number + 1 as filename
    output_path = os.path.join(OUTPUT_DIR, f"{i+1}.jpg")
    plt.savefig(output_path, dpi=300)
    plt.close()  # Close the figure to free memory
    print(f"Saved plot {i+1}.jpg to {OUTPUT_DIR}")